# CME 193 - Lecture 3 - SciPy

So far we've seen:
* Basic Python syntax
* Basic numeric arrays using NumPy

Today we'll see:
* Basic Python Classes
* Linear Algebra in SciPy

# Homework

Homework 1 is now posted.  It will be due in 1 week.  After today's class you'll have everything you need to complete it:
* Knowledge of Python Classes
* Know how to do some basic linear algebra in SciPy
* You've seen power method (last Lecture 2)

# Classes/Object Oriented Programming

The basic goal of classes is to give you a way to abstract away details when you program.  For instance, NumPy gave you an `array` class, which allowed you to store and pass around a large amount of information using a single variable, and perform a variety of operations on that variable (methods) without needing to implement them yourself.

Many programming languages have the ability to accomplish similar levels of abstraction, although not all use the same terminology.

Classes you've already seen:
* Integers
* Floats
* Lists
* NumPy Arrays
* PyPlot - Figures, Axes, etc.

When you want to make your own class, you need to write a class definition.  Then you can create objects using that class definition.

In [ ]:
# -- minimal example...
# define class:
class Leaf(object): 
    pass # there is no information in this class
# instantiate object
leaf = Leaf()

print(leaf)
print(type(leaf))
print(isinstance(leaf, Leaf)) # checks class membership

To put information in your class, add an initialization object

In [ ]:
# example
class Leaf(object):
    def __init__(self, color):
        self.color = color # private attribute 
    
redleaf = Leaf('red')
blueleaf = Leaf('blue')

print(redleaf.color) # access the attribute using .

## Class Hierarchy

A natural thing to think about as it relates to classes is the notion of *hierarchy*. We imbue the notion of hierarchy through something called *inheritance*.

An example:

* Animal
  * Bird
    * Hawk 
    * Seagull
    * ...
  * Dog
    * Shiba Inu
    * Golden Retriever
    * ...
  * ...

In [ ]:
# lets define an *abstract* base class.

class Animal(object):
    def __init__(self, n_legs, color):
        self.n_legs = n_legs 
        self.color = color
        
    def make_noise(self): 
        print('noise')

In [ ]:
# lets define some classes that will inherit
class Dog(Animal): # note we use (Animal), not (object)
    def __init__(self, color, name):
        Animal.__init__(self, 4, color) # 4 legs
        self.name = name 
        
    def make_noise(self):
        print(self.name + ': ' + 'woof')

In [ ]:
class Bird(Animal):
    def __init__(self, color, name, has_wings=True, can_fly=True):
        Animal.__init__(self, 2, color) # 2 legs
        self.name = name
        self.has_wings = has_wings
        self.can_fly = can_fly
    
    def make_noise(self):
        print(self.name + ': chirp!')

In [ ]:
# noise
brutus = Dog('black', 'Brutus')
brutus.make_noise()

In [ ]:
shelly = Bird('white', 'Shelly')
shelly.make_noise()

There are some standard methods that one may want to overload / implement:

* `__init__`: Constructor
* `__repr__`: Represent the object (machine) 
* `__str__`: Represent the object (human) and gets called when you `print`
    
these `__method__` looking functions are built into Python!

## Example: Rational Numbers

Here we'l make a class that holds rational numbers (fractions).  That is, numbers of the form
$$r = \frac{p}{q}$$
where $p$ and $q$ are integers

In [ ]:
from numpy import gcd

class Rational(object):
    def __init__(self, p, q=1):
    
        if q == 0:
            raise ValueError('Denominator must not be zero')
        if not isinstance(p, int):
            raise ValueError('Numerator must be an integer')
        if not isinstance(q, int):
            raise ValueError('Denominator must be an integer')
        
        g = int(gcd(p, q)) # because numpy returns a float
        
        self.p = p // g # integer division
        self.q = q // g
    
    # method to convert rational to float
    def __float__(self):
        return float(self.p) / float(self.q)    
    
    # method to convert rational to string for printing
    def __str__(self):
        return '%d / %d' % (self.p, self.q)

In [ ]:
a = Rational(6, 4)
b = Rational(3, 2)

print("a = ",a)
print("b = ",b)
print("float(a) = ", float(a))

You can do cool things like overload math operators.  This lets you write code that looks like you would write math.  Recall

$$ \frac{p_1}{q_1} + \frac{p_2}{q_2} = \frac{p_1 q_2 + p_2 q_1}{q_1 q_2}$$

In [ ]:
class Rational(object):
    def __init__(self, p, q=1):
    
        if q == 0:
            raise ValueError('Denominator must not be zero')
        if not isinstance(p, int):
            raise ValueError('Numerator must be an integer')
        if not isinstance(q, int):
            raise ValueError('Denominator must be an integer')
        
        g = int(gcd(p, q)) # because numpy returns a float
        
        self.p = p // g
        self.q = q // g
    
    # method to convert rational to float
    def __float__(self):
        return float(self.p) / float(self.q)    
    
    # method to convert rational to string for printing
    def __str__(self):
        return '%d / %d' % (self.p, self.q)
    
    # method to add two rationals
    def __add__(self, other): # that's two underscores "_" on each side
        '''
        this is R + X, where R is rational and X is another number
        '''
        if isinstance(other, Rational):
            return Rational(self.p * other.q + other.p * self.q, self.q * other.q)
        # -- if its an integer...
        elif isinstance(other, int):
            return Rational(self.p + other * self.q, self.q)
        # -- otherwise, we assume it will be a float
        return float(self) + float(other)
    
    def __radd__(self, other):
        '''
        this is X + R, where R is rational and X is not rational
        '''
        return self + other # addition commutes!
    

In [ ]:
r = Rational(3, 2)
print('Integer adding:')
print('right add')
print(r + 4)
print(float(r + 4))
print('left add')
print(4 + r)
print(float(4 + r))

# Exercise 1

### Add more operations to `Rational`
You can read about the available operations that you can overload [here](https://docs.python.org/3.3/reference/datamodel.html#emulating-numeric-types)

Add the following operations to the `Rational` class:
* `*` - use `__mul__` (and `__rmul__`)
* `/` - use `__truediv__` (and `__rtruediv__`)
* `-` - use `__sub__` (and `__rsub__`)

You only need to define these operations between two `Rational` types - use an `isinstance(other, Rational)` block.

Note that the `__r<op>__` methods only need to be overloaded if you want to interact with another class.

Make a few examples to convince yourself that this works.

### Create another class

Implement a class to do arithmetic in the ring $\mathbb{Z} \mod N$ for some $N > 1$:
* $a + b = (a + b) \mod N$
* $a * b = (a * b) \mod N$

You can either:
* Choose a value for $N$ and hard-code it into your class
* Specify $N$ for each object you create.
    * make sure two objects are in the same ring when you do arithmetic!

# SciPy

SciPy is a library with a variety of modules useful for scientific computing.

* Linear Algebra (Dense, Sparse)
* Optimization
* Special Functions
* Integration
* Image Processing
* Statistics
* ...

See the [online reference](https://docs.scipy.org/doc/scipy/reference/) for additional information.

The library is built on NumPy arrays for dense vectors/matrices.

Today, we'll cover some linear algebra capabilities.  Thursday, we'll see the optimization module.

# SciPy Dense Linear Algebra

Why have a SciPy linear algebra module, when NumPy already has this?

* SciPy always uses BLAS/LAPACK under the hood (usually faster)
* SciPy adds to the number of functions available.

You can find an introductory reference [here](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html).  The syntax is the same as numpy for functions that are in both.

In [ ]:
import scipy.linalg as sla
import numpy.linalg as nla

In [ ]:
# example of SVD
A = np.random.normal(0, 1, (2,3))
U, S, V = sla.svd(A) # scipy
print("A  = \n", A)
print("U = \n", U)
print("S = \n", S)
print("V = \n", V)
U, S, V = nla.svd(A) # numpy
print("U = \n", U)
print("S = \n", S)
print("V = \n", V)

# Sparse Linear Algebra

We've seen a bit of dense linear algebra in both NumPy and SciPy.  If you want to do sparse linear algebra, SciPy's `sparse` module provides standard sparse matrix formats, and functionality to solve linear systems, find eigenvalues, etc.

[SciPy's Sparse Matrix Docs](https://docs.scipy.org/doc/scipy/reference/sparse.html)

[SciPy's Sparse Linear Algebra Docs](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html#module-scipy.sparse.linalg)

SciPy supports a variety of sparse matrix types:
* CSC (compressed sparse column)
* CSR (compressed sparse row)
* COO (coordinate)
* LIL (linked lists)
* ... (specialized matrix types)

[SciPy's Sparse Matrix Classes](https://docs.scipy.org/doc/scipy/reference/sparse.html#sparse-matrix-classes)

In [ ]:
import numpy as np
import scipy.sparse as sparse

## Example: A Diffusion Operator

suppose we want to construct a matrix representation of the laplacian
$$ \Delta = \frac{\partial^2}{\partial_x^2}$$
We'll stick to one dimension for simplicity, and use a simple finite difference scheme with spacing $h$
$$ (\Delta u)_i = \frac{1}{h^2}(-2 u_i + u_{i+1} + u_{i-1})$$
We'll use the matrix `L` to hold our Laplacian
```python
(L * u)[i] = (u[i+1] + u[i-1] - 2*u[i]) / h^2
```
The matrix `L` looks like
$$\frac{1}{h^2}\begin{bmatrix}
\ddots &1 \\
1 &-2 & 1\\
& 1 & -2 & 1\\
& & 1&\ddots
\end{bmatrix}$$
Note that I haven't said anything about boundary conditions...

We'll now construct this matrix using `scipy.sparse.spdiags` which allows us to construct a sparse matrix with given diagonals - [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.spdiags.html#scipy.sparse.spdiags)

In [ ]:
n = 100
h = 1.0/n
# construct diagonal
diag = -2*np.ones(n)
diag[0] = -1.
diag[-1] = -1.
diag /= (h**2)
# construct off-diagonal
odiag = np.ones(n) / (h**2)
data = np.array([odiag, diag, odiag]) # entries
diags = np.array([-1, 0, 1]) # left of diagonal, diagonal, right of diagonal
L = sparse.spdiags(data, diags, n, n)
L

In [ ]:
L.toarray()

In [ ]:
u = np.random.normal(0, 1, n)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5,4))

ax.plot(np.linspace(0,1,n), u)
ax.set_title(r"$u$")
plt.show()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5,4))

ax.plot(np.linspace(0,1,n), L.dot(u))
ax.set_title(r"$\Delta u$")
plt.show()

Now we'll run the heat equation for a few time steps
$$\frac{\partial u}{\partial t} - \Delta_x u = 0$$
We'll take time steps of length $O(h^2)$.

In [ ]:
u_smoothed = u.copy()

In [ ]:
for i in range(1000):
    ut = L.dot(u_smoothed)
    u_smoothed += (ut * (h**2)/4)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5,4))

ax.plot(np.linspace(0,1,n), u_smoothed)
ax.set_title(r"after heat diffusion")
plt.show()

## Solving The Poisson Equation

Now, suppose we want to solve the Poisson equation
$$ - \Delta u = f $$
Since $-\Delta$ is symmetric positive, definite, we can use the conjugate gradient (`cg`) algorithm to solve the system.  Alternatively, we can use `minres`.

You can see available iterative methods [here](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html#solving-linear-problems)

In [ ]:
def u_gen(x):
    return 0.25-np.square(x - 0.5)
    
u_true = u_gen(np.linspace(0,1,n))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,4))

ax.plot(np.linspace(0,1,n), u_true)
ax.set_title(r"True solution")
plt.show()

In [ ]:
f = -L.dot(u_true)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,4))

ax.plot(np.linspace(0,1,n), f)
ax.set_title(r"$f = -\Delta u$")
plt.show()

In [ ]:
u_minres, info = sparse.linalg.minres(-L, f)
u_cg, info = sparse.linalg.cg(-L, f)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,4))

ax.plot(np.linspace(0,1,n), u_cg, label="cg sol")
ax.plot(np.linspace(0,1,n), u_minres, label="minres sol")
ax.plot(np.linspace(0,1,n), u_true, label="true")
ax.legend()
ax.set_title(r"Poisson Equation")
plt.show()

Why do the plots disagree?

Note that a constant vector is in the nullspace of $-\Delta$, so without specifying boundary conditions, both solutions are correct.

# Exercise 2

Eigenvectors (eigenfunctions) of the Laplacian $\Delta$ tell us about the vibrational modes of an object.  So far we've constructed a discrete approxim Laplacian on a unit-length "string"

1. Use [`eigsh`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html#scipy.sparse.linalg.eigsh) to compute the smallest (by magnitude of eigenvalue) 4 eigenpairs of $\Delta$.  Plot the eigenvectors in a single plot.
    1. Hint: use the keyword `which='SM'` to get smallest magnitude eigenvalues
2.  Construct a finite difference laplacian on the unit square (100 grid points in each dimension), and compute the smallest several eigenpairs
    1. You can construct the matrix explicitly, or use [`kron`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.kron.html#scipy.sparse.kron)
    2. Hint: if you use `kron`, $L_2 = L_1\otimes I + I \otimes L_1$, where $L_2$ is the laplacian on the square, and $L_1$ is the laplacian on the interval
    3. visualize the eigenvectors as 2-D images
    4. Hint: use `numpy.reshape` and `plt.imshow`
3. In parts 1 and/or 2 What do the eigenvectors with largest eigenvalue look like? (Look at documentation for the `which` keyword again)